# Тестирование Spell Checker с использованием дополнительных признаков

В этом ноутбуке будет произведено тестирование модели, в которой в candidate scorer используются модели на основе множества признаков помимо BERT.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gc
import sys
import os
import json
import pickle
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from sklearn.svm import LinearSVC

from IPython.display import display
from tqdm.notebook import tqdm

In [ ]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [ ]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [ ]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

In [ ]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
margin_border = np.log(2.5)
position_selector = KenlmMarginPositionSelector(
    model_left_right, model_right_left, margin_border=margin_border
)

In [ ]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = 'mean'
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)

with open(os.path.join(MODEL_PATH, 'candidate_scorer', 'svm.bin'), 'rb') as inf:
    svm_model = pickle.load(inf)

svm_scorer = SVMScorer(svm_model, bert_scorer=bert_scorer)
candidate_scorer = CandidateScorer(svm_scorer)

In [ ]:
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    tokenizer,
    detokenizer,
    ignore_titles=True,
    max_it=max_it,
    combine_tokens=False
)

## Тестирование

### Обучающая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
!mkdir -p ../data/processed/results_svm/

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'train.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_train.txt ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results_svm/train.txt | tail -n 4

**Ranking SVM**

* True Positive: $1305$
* Внесенных исправлений: $1446$ 
* Требуемых исправления: $1727$ 
* Precision: $90.25$, доверительный интервал: $(88.72, 91.78)$
* Recall: $75.56$, доверительный интервал: $(74.28, 76.85)$
* FMeasure: $82.26$, доверительный интервал: $(80.86, 83.65)$

**Ranking SVM (без BERT)**

* True Positive: $1270$
* Внесенных исправлений: $1428$ 
* Требуемых исправления: $1727$ 
* Precision: $88.94$, доверительный интервал: $(87.32, 90.56)$
* Recall: $73.54$, доверительный интервал: $(72.20, 74.88)$
* FMeasure: $80.51$, доверительный интервал: $(79.04, 81.97)$

**Ranking SVM (с объединением токенов)**

* True Positive: $1331$
* Внесенных исправлений: $1484$ 
* Требуемых исправления: $1727$ 
* Precision: $89.69$, доверительный интервал: $(88.15, 91.23)$
* Recall: $77.07$, доверительный интервал: $(75.74, 78.40)$
* FMeasure: $82.90$, доверительный интервал: $(81.48, 84.33)$

**Ranking SVM (с объединением токенов, без BERT)**

* True Positive: $1292$
* Внесенных исправлений: $1455$ 
* Требуемых исправления: $1727$ 
* Precision: $88.80$, доверительный интервал: $(87.18, 90.41)$
* Recall: $74.81$, доверительный интервал: $(73.45, 76.17)$
* FMeasure: $81.21$, доверительный интервал: $(79.73, 82.68)$

**CatBoost**

* True Positive: $1353$
* Внесенных исправлений: $1463$ 
* Требуемых исправления: $1727$ 
* Precision: $92.48$, доверительный интервал: $(91.13, 93.83)$
* Recall: $78.34$, доверительный интервал: $(77.20, 79.49)$
* FMeasure: $84.83$, доверительный интервал: $(83.59, 86.07)$

**CatBoost (без BERT)**

* True Positive: $1315$
* Внесенных исправлений: $1452$ 
* Требуемых исправления: $1727$ 
* Precision: $90.56$, доверительный интервал: $(89.06, 92.07)$
* Recall: $76.14$, доверительный интервал: $(74.88, 77.41)$
* FMeasure: $82.73$, доверительный интервал: $(81.36, 84.10)$

**CatBoost (с объединением токенов)**

* True Positive: $1385$
* Внесенных исправлений: $1495$ 
* Требуемых исправления: $1727$ 
* Precision: $92.64$, доверительный интервал: $(91.32, 93.97)$
* Recall: $80.20$, доверительный интервал: $(79.05, 81.34)$
* FMeasure: $85.97$, доверительный интервал: $(84.74, 87.20)$

**CatBoost (с объединением токенов, без BERT)**

* True Positive: $1352$
* Внесенных исправлений: $1485$ 
* Требуемых исправления: $1727$ 
* Precision: $91.04$, доверительный интервал: $(89.59, 92.50)$
* Recall: $78.29$, доверительный интервал: $(77.03, 79.54)$
* FMeasure: $84.18$, доверительный интервал: $(82.84, 85.53)$

### Тестовая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'test_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'test_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_test.txt ../data/external/spell_ru_eval/test_source.txt ../data/external/spell_ru_eval/test_corrected.txt ../data/processed/results_svm/test.txt | tail -n 4

**Ranking SVM**

* True Positive: $1373$
* Внесенных исправлений: $1590$ 
* Требуемых исправления: $1976$ 
* Precision: $86.35$, доверительный интервал: $(84.66, 88.04)$
* Recall: $69.48$, доверительный интервал: $(68.12, 70.84)$
* FMeasure: $77.01$, доверительный интервал: $(75.50, 78.51)$

**Ranking SVM (без BERT)**

* True Positive: $1323$
* Внесенных исправлений: $1559$ 
* Требуемых исправления: $1976$
* Precision: $84.86$, доверительный интервал: $(83.08, 86.64)$
* Recall: $66.95$, доверительный интервал: $(65.55, 68.36)$
* FMeasure: $74.85$, доверительный интервал: $(73.28, 76.42)$

**Ranking SVM (с объединением токенов)**

* True Positive: $1430$
* Внесенных исправлений: $1656$ 
* Требуемых исправления: $1976$ 
* Precision: $86.35$, доверительный интервал: $(84.71, 88.00)$
* Recall: $72.37$, доверительный интервал: $(70.99, 73.75)$
* FMeasure: $78.74$, доверительный интервал: $(77.24, 80.25)$

**Ranking SVM (с объединением токенов, без BERT)**

* True Positive: $1375$
* Внесенных исправлений: $1624$ 
* Требуемых исправления: $1976$
* Precision: $84.67$, доверительный интервал: $(82.91, 86.42)$
* Recall: $69.59$, доверительный интервал: $(68.14, 71.03)$
* FMeasure: $76.39$, доверительный интервал: $(74.81, 77.97)$

**CatBoost**

* True Positive: $1375$
* Внесенных исправлений: $1579$ 
* Требуемых исправления: $1976$
* Precision: $87.08$, доверительный интервал: $(85.42, 88.74)$
* Recall: $69.59$, доверительный интервал: $(68.26, 70.91)$
* FMeasure: $77.36$, доверительный интервал: $(75.88, 78.83)$

**CatBoost (без BERT)**

* True Positive: $1336$
* Внесенных исправлений: $1564$ 
* Требуемых исправления: $1976$
* Precision: $85.42$, доверительный интервал: $(83.67, 87.18)$
* Recall: $67.61$, доверительный интервал: $(66.22, 69.00)$
* FMeasure: $75.48$, доверительный интервал: $(73.93, 77.03)$

**CatBoost (с объединением токенов)**

* True Positive: $1430$
* Внесенных исправлений: $1652$ 
* Требуемых исправления: $1976$
* Precision: $86.56$, доверительный интервал: $(84.92, 88.21)$
* Recall: $72.37$, доверительный интервал: $(70.99, 73.74)$
* FMeasure: $78.83$, доверительный интервал: $(77.33, 80.33)$

**CatBoost (с объединением токенов, без BERT)**

* True Positive: $1399$
* Внесенных исправлений: $1640$ 
* Требуемых исправления: $1976$
* Precision: $85.30$, доверительный интервал: $(83.60, 87.01)$
* Recall: $70.80$, доверительный интервал: $(69.38, 72.22)$
* FMeasure: $77.38$, доверительный интервал: $(75.83, 78.93)$

## Выводы

1. Результаты удалось заметно улучшить и достичь SOTA.
2. Модели без признаков BERT показывают результат на $\approx 2\%$ хуже.
3. Добавление объединения токенов улучшает результат на $\approx 1\%$.